In [1]:
from helpers import fetch_data
import polars as pl
import pandas as pd
from polars import col as c
import polars as pl
from polars import col as c
import polars.selectors as cs
import plotly.express as px

In [14]:


date_id = 120
state_filter = 'XX'  # Example state filter, replace with actual state code if needed
df = fetch_data(date_id, 'XX', 'product_group').collect()
df.to_pandas()

,product_group,units,rx_count,total_diff,new_nadac,old_nadac,avg_new_nadac,avg_old_nadac,diff_per_rx,total_diff_abs,diff_per_rx_abs,classification
0,Antiemetic Combination - Two Ingredient,1.343382e+06,21542.0,-1310680,2944371.26,4255053.21,2.1918,3.1674,-60.84,1310680,60.84,Decrease
1,Zonisamide,4.313824e+07,385097.0,-418120,4716258.24,5134384.86,0.1093,0.1190,-1.09,418120,1.09,Decrease
2,Benzonatate,7.422220e+07,2391909.0,-957964,7184190.76,8142166.17,0.0968,0.1097,-0.40,957964,0.40,Decrease
3,Haloperidol,2.832219e+07,571908.0,-1343743,10790166.42,12133921.08,0.3810,0.4284,-2.35,1343743,2.35,Decrease
4,Cilostazol,6.937953e+06,101309.0,-38275,859251.49,897529.19,0.1238,0.1294,-0.38,38275,0.38,Decrease
...,...,...,...,...,...,...,...,...,...,...,...,...
441,Fosinopril,4.092250e+05,7923.0,-6619,70704.38,77325.74,0.1728,0.1890,-0.84,6619,0.84,Decrease
442,Continuous Contraceptives Oral - Two Ingredient,2.732204e+05,5389.0,-23951,339099.26,363052.31,1.2411,1.3288,-4.44,23951,4.44,Decrease
443,Albuterol,2.219545e+05,2682.0,-330314,69391.86,399707.65,0.3126,1.8009,-123.16,330314,123.16,Decrease
444,Platelet Aggregation Inhibitor Combination - T...,2.768080e+05,3938.0,-70952,221006.27,291958.78,0.7984,1.0547,-18.02,70952,18.02,Decrease


In [34]:

def scatter_plot(df):
    fig = px.scatter(
        df,
        x='rx_count',
        y='total_diff_abs',
        size='total_diff_abs',
        color='classification',
        hover_data={'rx_count': True, 'total_diff': ':$,.0f', 'diff_per_rx': ':$,.2f'},
        labels={
            'rx_count': 'Prescription Count',
            'total_diff': 'Total Difference ($)',
            'diff_per_rx': 'Difference per Rx ($)'
        },
        title='Prescription Analysis: Total Difference vs Prescription Count',
        log_x=True,
        log_y=True,
        size_max=40,
    )

    # Update layout for professional appearance and custom x-axis tick formatting
    fig.update_layout(
        title={
            'text': 'Prescription Analysis: Total Difference vs Prescription Count',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 18, 'family': 'Arial, sans-serif'}
        },
        xaxis={
            'title': {'text': 'Prescription Count (log scale)', 'font': {'size': 14}},
            'tickformat': '~s',  # SI prefix, e.g., 1K, 1M
            'gridcolor': 'lightgray',
            'gridwidth': 0.5
        },
        yaxis={
            'title': {'text': 'Total Difference ($, log scale)', 'font': {'size': 14}},
            'tickformat': '$~s',  # SI prefix with dollar sign, e.g., $1K, $1M
            'gridcolor': 'lightgray',
            'gridwidth': 0.5
        },
        coloraxis_colorbar={
            'title': {'text': 'Total Difference ($)', 'font': {'size': 12}},
            'tickformat': '$~s'  # SI prefix with dollar sign
        },
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'family': 'Arial, sans-serif', 'size': 12},
        margin={'l': 80, 'r': 80, 't': 100, 'b': 80},
        width=1200,
        height=600
    )

    # Update traces for better appearance
    fig.update_traces(
        marker={
            'sizemin': 5,
            'line': {'width': 0.5, 'color': 'white'},
            'opacity': 0.8
        },
        hovertemplate=
            '<b>Product:</b> %{customdata[0]}<br>' +
            '<b>Prescription Count:</b> %{x:,}<br>' +
            '<b>Total Difference:</b> $%{y:,.0f}<br>' +
            '<b>Difference per Rx:</b> $%{customdata[1]:,.2f}<br>' +
            '<extra></extra>',
        customdata=df[['product_group', 'diff_per_rx']].to_numpy()
    )
    return fig

scatter_plot(df)
